# LMER in rpy2

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from CrossOverExperiment import CrossOverExperiment, RCT

import rpy2

# Install LMER packages (THIS TAKES ABOUT 3~5 minutes)
packnames = ('lme4', 'lmerTest', 'emmeans', "geepack")
from rpy2.robjects.packages import importr, data
from rpy2.robjects.vectors import StrVector

utils = importr("utils")
utils.chooseCRANmirror(ind=1)

import pandas as pd
import rpy2.robjects as robjects
from rpy2.ipython.ggplot import image_png
from rpy2.robjects import pandas2ri# Defining the R script and loading the instance in Python
r = robjects.r

import numpy as np
import pandas as pd

pandas2ri.activate()

import statsmodels.api as sm

### I had to install all these in this order before getting lme4 to work with rpy2

## R imports

In [3]:
lme4 = importr('lme4')
lmertest = importr('lmerTest')
base = importr('base')
stats = importr("stats")

## prepare some test data

In [4]:
crossover_experiment = CrossOverExperiment(subjects=15,periods=6,base_effect=2,mu=1,tau=0.2,carryover=0.1,alpha_sd=2,epsilon_sd=1,error_type="whatever, statsmodels supports only idiosyncratic")
crossover_experiment.params

{'Subjects': 15,
 'Periods': 6,
 'Base Effect': 2,
 'Mu': 1,
 'Tau': 0.2,
 'Carryover Effect': 0.1,
 'Alpha Standard Deviation': 2,
 'Epsilon Standard Deviation': 1,
 'Error Structure': 'whatever, statsmodels supports only idiosyncratic'}

In [5]:
(y0,y1),sub,T,t,cO = crossover_experiment.generate_data(return_for_t=False)

In [6]:
exog = pd.DataFrame(np.array([np.ones(len(T)),T,t,cO,sub]).T)#.reset_index()
exog.columns = ["len_T", "T", "t", "c0", "sub"]

endog = pd.DataFrame(y0)
endog.columns = ["y0"]

endogexog = pd.concat([endog,exog], axis=1)

### statsmodels mlm

In [7]:
mlm0 = sm.MixedLM(endog=endog,exog=exog.drop(columns="sub"),groups=sub).fit()#.pvalues[1]

In [8]:
mlm0.pvalues

len_T        0.002307
T            0.611383
t            0.000554
c0           0.880562
Group Var    0.020529
dtype: float64

In [9]:
!pip freeze | grep statsmodels
# statsmodels==0.13.2
# installing older versions did not erase the error

statsmodels==0.13.2


In [10]:
mlm0.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
=======================================================
Model:            MixedLM Dependent Variable: y0       
No. Observations: 90      Method:             REML     
No. Groups:       15      Scale:              0.9362   
Min. group size:  6       Log-Likelihood:     -149.1819
Max. group size:  6       Converged:          Yes      
Mean group size:  6.0                                  
--------------------------------------------------------
           Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
--------------------------------------------------------
len_T      1.683     0.552  3.048  0.002   0.601   2.766
T          0.116     0.228  0.508  0.611  -0.331   0.562
t          0.213     0.062  3.453  0.001   0.092   0.333
c0         0.036     0.241  0.150  0.881  -0.435   0.508
Group Var  3.461     1.544                              
=======================================================

"""

## try lmertest

In [11]:
fm2 = lmertest.lmer('y0 ~ (1|sub) + len_T + T + t + c0', data = endogexog)
print(base.summary(fm2))

R[write to console]: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: "y0 ~ (1|sub) + len_T + T + t + c0"
   Data: 
structure(list(y0 = c(1.6045185432128, 0.00206142334413695, -0.359708740947737,  
-0.447335875847201, -1.02852293956131, 0.609436887251567, 4.00878240811743,  
5.1221806207631, 5.60033195851215, 4.39542446208106, 4.05869602330635,  
4.22250446597168, 1.21260435564816, 1.44764886721966, 0.974223976023323,  
3.31784163742145, 1.59239194339443, 3.29586523328598, 0.392633174391802,  
1.03408547279356, 0.669076077136336, 1.31972104898138, -0.307485669296077,  
3.90527971937734, 1.12634176626808, 1.70607643907361, 1.94344085783685,  
4.44712950449553, 2.69741713206544, 2.32756487141417, 1.68839390404697,  
1.92765482223205, 3.62700008060025, 1.74669947770604, 2.50420431466707,  
2.39983709472747, 3.21208279555628, 0.273468819427187, 2.64168421923298,  
1.50605660138299, 1.87256445747701, 0.678172596445647, 1.88087996363214,  
1.82912634109268, 1.4499017

## try to extract p-values

In [12]:
fm2 = lmertest.lmer('y0 ~ (1|sub) + len_T + T + t + c0', data = endogexog)
print(stats.anova(fm2))

R[write to console]: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Type III Analysis of Variance Table with Satterthwaite's method
       Sum Sq Mean Sq NumDF  DenDF F value    Pr(>F)    
T      0.2417  0.2417     1 72.051  0.2582 0.6129340    
t     11.1633 11.1633     1 72.017 11.9240 0.0009318 ***
c0     0.0211  0.0211     1 72.242  0.0226 0.8809805    
len_T                                                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



In [13]:
fm2anova = stats.anova(fm2)

In [14]:
fm2anova.rx2("Pr(>F)")[0] # p_values

0.6129340390657186

In [15]:
fm2anova.rx2("Pr(>F)") # p_values

array([0.61293404, 0.00093177, 0.88098045,        nan])

In [16]:
mlm0.pvalues

len_T        0.002307
T            0.611383
t            0.000554
c0           0.880562
Group Var    0.020529
dtype: float64

In [17]:
mlm0.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
=======================================================
Model:            MixedLM Dependent Variable: y0       
No. Observations: 90      Method:             REML     
No. Groups:       15      Scale:              0.9362   
Min. group size:  6       Log-Likelihood:     -149.1819
Max. group size:  6       Converged:          Yes      
Mean group size:  6.0                                  
--------------------------------------------------------
           Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
--------------------------------------------------------
len_T      1.683     0.552  3.048  0.002   0.601   2.766
T          0.116     0.228  0.508  0.611  -0.331   0.562
t          0.213     0.062  3.453  0.001   0.092   0.333
c0         0.036     0.241  0.150  0.881  -0.435   0.508
Group Var  3.461     1.544                              
=======================================================

"""

In [18]:
print(stats.anova(fm2))

Type III Analysis of Variance Table with Satterthwaite's method
       Sum Sq Mean Sq NumDF  DenDF F value    Pr(>F)    
T      0.2417  0.2417     1 72.051  0.2582 0.6129340    
t     11.1633 11.1633     1 72.017 11.9240 0.0009318 ***
c0     0.0211  0.0211     1 72.242  0.0226 0.8809805    
len_T                                                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



In [19]:
formula = robjects.Formula('y0 ~ T + t + c0')
random = robjects.Formula('~1|sub')

In [22]:
# apparently you can provide a correlation= argument here for the AR1

nlme = importr("nlme")
m1 = nlme.lme(formula,random=random,data=endogexog)

In [23]:
print(stats.anova(m1))

            numDF denDF   F-value p-value
(Intercept)     1    72 25.911913  <.0001
T               1    72  0.220223  0.6403
t               1    72 12.938104  0.0006
c0              1    72  0.022577  0.8810

